In [2]:
print("hello2")

hello2


In [30]:
import csv
import subprocess
import getpass
import os
import gzip
from os import listdir
from os.path import isfile, join
from SciServer import Authentication

myUserName = Authentication.getKeystoneUserWithToken(Authentication.getToken()).userName
passwd = getpass.getpass('Password for ' + myUserName + ': ')
userstring = "username=" + myUserName + ",workgroup=win,uid=idies,password=" + passwd
projectname = input("Please enter the name of your SAFE Desktop project folder : ") 
dir = "//cloud.nas.jh.edu/sddesktop$/" + projectname
devnull = open(os.devnull, 'w')
subprocess.run(["sudo", "mkdir", "/home/idies/workspace/SAFE"], capture_output=False)
subprocess.run(["sudo", "chown", "idies:idies", "/home/idies/workspace/SAFE/"], capture_output=False)
try:
    subprocess.run(["sudo", "mount", "-t", "cifs", dir, "/home/idies/workspace/SAFE/", "-o", userstring], stdout=devnull, stderr=devnull)
except FileNotFoundError as e:
    print(e)

Password for mkim239:  ··········
Please enter the name of your SAFE Desktop project folder :  LCICM


In [17]:
# extracting patient_id of patients that have ECG recordings
# UPDATE: changing to patients that have 12-lead ECG recordings
patient_groups = [ f.path for f in os.scandir('SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/') if f.is_dir ]

In [4]:
pip install wfdb pandas pyarrow

Note: you may need to restart the kernel to use updated packages.


In [5]:
import wfdb
import pandas as pd
import pyarrow
print("import successful")

import successful


In [11]:
def make_parquet(record_name, directory, name):
    # ex record name: 'SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/p1000/p10000032/s40689238/40689238'
    record = wfdb.rdrecord(record_name)
    ecg_data = record.p_signal
    lead_names = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
    ecg_df = pd.DataFrame(data=ecg_data, columns=lead_names)    
    
    # create diectory for parquet if it doesn't exist
    os.makedirs(directory, exist_ok = True)
    
    # save parquet in the directory
    ecg_df.to_parquet(directory+name, index=False, engine='pyarrow')

In [63]:
def get_time_ecg(file_path):
    with open(file_path, 'r', encoding = 'utf-8') as file:
        # get time
        lines = file.readlines()
        first_line = lines[0].split()
        time = first_line[-2:]
        time = '_'.join(time)
        time = time.replace("/", "_")
        
        # get patient id
        last_line = lines[-1].split()
        patient_id = last_line[-1]
        full_name = time + "_" + patient_id + ".parquet"
        
        # TODO:
        # ADD FILE PATH FOR WHERE TO STORE PARQUET HERE:
        full_path = 'SAFE/Databases/mimic-iv-ecg-parquet/' + patient_id + '/'
        return full_path, full_name
    

In [59]:
print(get_time_ecg('SAFE/Databases/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/files/p1000/p10000032/s40689238/40689238.hea'))

('SAFE/Databases/mimic-iv-ecg-parquet/10000032/', '08:44:00_23_07_2180_10000032.parquet')


In [114]:
# basic ECG count & extracting the full list of patient IDs
ecg_ids = []
for patient_group in patient_groups:
    for f in os.scandir(patient_group):
        if f.is_dir():
            ecg_ids.append(int(f.name[1:]))
print(len(ecg_ids))

160720


In [ ]:
#19414987
# store every 12-lead into parquet format in S drive
# every patient group
for patient_group in patient_groups:
    # every patient
    for f in os.scandir(patient_group):
        if f.is_dir():
            # every study id
            for f2 in os.scandir(f):
                if f2.is_dir():
                    raw_files = os.listdir(f2)
                    # remove junk file
                    files = [item for item in raw_files if item != '.ipynb_checkpoints']
                    print(files)
                    
                    # 0 => .dat file 
                    # 1 => .hea fil
                    dat = ""
                    hea = ""
                    for file_name in files:
                        if file_name[-4:] == ".hea":
                             hea = os.path.join(f2, file_name)
                        if file_name[-4:] == ".dat":
                             dat = os.path.join(f2, file_name)
                    parquet_name = get_time_ecg(hea)
                    print(parquet_name[0])
                    print(parquet_name[1])
                    make_parquet(dat[:-4], parquet_name[0], parquet_name[1])
                    print("Added: " + parquet_name[1])

['41131947.dat', '41131947.hea']
SAFE/Databases/mimic-iv-ecg-parquet/19414987/
09:49:00_19_03_2184_19414987.parquet
Added: 09:49:00_19_03_2184_19414987.parquet
['42770247.dat', '42770247.hea']
SAFE/Databases/mimic-iv-ecg-parquet/19414987/
15:26:00_10_10_2185_19414987.parquet
Added: 15:26:00_10_10_2185_19414987.parquet
['47853000.hea', '47853000.dat']
SAFE/Databases/mimic-iv-ecg-parquet/19414987/
08:51:00_18_03_2184_19414987.parquet
Added: 08:51:00_18_03_2184_19414987.parquet
['48180945.dat', '48180945.hea']
SAFE/Databases/mimic-iv-ecg-parquet/19414987/
10:06:00_14_12_2191_19414987.parquet
Added: 10:06:00_14_12_2191_19414987.parquet
['44546665.dat', '44546665.hea']
SAFE/Databases/mimic-iv-ecg-parquet/19414987/
08:26:00_20_03_2184_19414987.parquet
Added: 08:26:00_20_03_2184_19414987.parquet
['49800992.hea', '49800992.dat']
SAFE/Databases/mimic-iv-ecg-parquet/19414987/
19:29:00_01_03_2190_19414987.parquet
Added: 19:29:00_01_03_2190_19414987.parquet
['49931470.dat', '49931470.hea']
SAFE/Da